In [4]:
"""
Consolidate Markdown Tables: Merge split tables across pages

Algorithm:
- Find table blocks (header line followed by separator |---|...)
- For consecutive table blocks, if headers match and columns align, merge them
- For single contiguous blocks with repeated headers inside, remove the duplicate headers
- Only processes markdown files (post-extraction consolidation)
"""
import re
from pathlib import Path

TABLE_HEADER_RE = re.compile(r"^\s*\|.*\|\s*$")
SEPARATOR_RE = re.compile(r"^\s*\|(?:\s*-+\s*\|)+\s*$")

def normalize_row(row):
    # Normalize header row for comparison: lowercase, collapse whitespace
    return re.sub(r"\s+", " ", row.strip().lower())

def find_table_blocks(lines):
    blocks = []  # list of (start_idx, end_idx)
    i = 0
    n = len(lines)
    while i < n:
        if TABLE_HEADER_RE.match(lines[i]):
            # candidate header, check next line is separator
            if i + 1 < n and SEPARATOR_RE.match(lines[i+1]):
                start = i
                j = i + 2
                # extend until a non-table line
                while j < n and lines[j].strip().startswith('|'):
                    j += 1
                end = j - 1
                blocks.append((start, end))
                i = j
                continue
        i += 1
    return blocks

## Deduplicate & Merge Table Blocks

**dedupe_duplicate_headers_within_block()**:
- Removes repeated header+separator pairs within a single table block
- Happens when PDF has headers that repeat due to page breaks
- Returns: (deduplicated_lines, changed_flag)

**merge_blocks_in_lines()**:
- Merges consecutive table blocks with matching headers and column count
- Only merges if intermediate lines are blank (no content between tables)
- Iteratively processes until no more merges are possible
- Returns: True if any merges occurred

In [5]:
def dedupe_duplicate_headers_within_block(block_lines):
    """
    Given a list of lines that form a contiguous markdown table block,
    remove any repeated header+separator pairs that appear later in the block.
    """
    if len(block_lines) < 3:
        return block_lines, False

    header = normalize_row(block_lines[0])
    changed = False
    i = 2  # start after header and separator
    out_lines = block_lines[:2]
    while i < len(block_lines):
        line = block_lines[i]
        # if this line looks like the header and next is a separator -> skip both
        next_line = block_lines[i+1] if i+1 < len(block_lines) else ''
        if normalize_row(line) == header and SEPARATOR_RE.match(next_line):
            # skip header+separator
            changed = True
            i += 2
            # continue without adding these lines
            continue
        out_lines.append(line)
        i += 1

    return out_lines, changed

def merge_blocks_in_lines(lines):
    changed = False
    blocks = find_table_blocks(lines)
    # iterate and attempt merges
    i = 1
    while i < len(blocks):
        prev_start, prev_end = blocks[i-1]
        cur_start, cur_end = blocks[i]
        # check intermediate lines between prev_end and cur_start are blank
        inter = ''.join(lines[prev_end+1:cur_start]).strip()
        if inter != '':
            i += 1
            continue
        # get header rows
        header_prev = normalize_row(lines[prev_start])
        header_cur = normalize_row(lines[cur_start])
        if header_prev != header_cur:
            i += 1
            continue
        # count columns by splitting header on | and ignoring empty edges
        def col_count(row):
            parts = [p.strip() for p in row.strip().strip('|').split('|')]
            return len(parts)
        if col_count(lines[prev_start]) != col_count(lines[cur_start]):
            i += 1
            continue
        # perform merge: append data rows from cur (skip header and separator) to prev
        data_rows = lines[cur_start+2:cur_end+1]
        # remove trailing empty lines from prev_end if any
        insert_pos = prev_end + 1
        # insert data rows
        lines[insert_pos:insert_pos] = data_rows
        # now remove the original cur block (which shifted forward by len(data_rows))
        del lines[insert_pos+len(data_rows): insert_pos+len(data_rows) + (cur_end - cur_start +1)]
        changed = True
        # rebuild blocks and restart scanning
        blocks = find_table_blocks(lines)
        i = 1
    return changed

## Process Files & Execute Consolidation

**process_file()**:
- Main consolidation processor for a single markdown file
- Merges consecutive table blocks first
- Then dedupes headers within each block
- Writes changes back to file if any modifications occurred
- Returns: True if file was modified

**consolidate_md_tables()**:
- Entry point: processes all markdown files in a directory
- Calls process_file() on each markdown file
- Tracks and reports total files changed
- Returns: count of files that were modified

In [6]:
def process_file(path: Path):
    text = path.read_text(encoding='utf-8')
    lines = text.splitlines(keepends=True)
    changed = merge_blocks_in_lines(lines)
    # Also dedupe repeated header+separator pairs that may appear inside a single block
    blocks = find_table_blocks(lines)
    for start, end in reversed(blocks):
        block_lines = lines[start:end+1]
        new_block, block_changed = dedupe_duplicate_headers_within_block(block_lines)
        if block_changed:
            lines[start:end+1] = new_block
            changed = True
    if changed:
        path.write_text(''.join(lines), encoding='utf-8')
    return changed

def consolidate_md_tables(dir_path):
    """Process all markdown files in directory to consolidate split tables."""
    target = Path(dir_path)
    files = list(target.rglob('*.md')) if target.is_dir() else [target] if target.is_file() else []
    
    total_changed = 0
    for f in files:
        changed = process_file(f)
        if changed:
            print(f'✓ Merged tables in: {f.name}')
            total_changed += 1
    
    return total_changed

## Execute Table Consolidation

Run the consolidation process on all generated markdown files to merge split tables and remove duplicate headers.

In [8]:
import os
import re

def find_tables_in_markdown_files(directory):
    """
    Scan all markdown files in a directory and report where tables are found.
    """
    table_pattern = re.compile(r"^\s*\|\s*.*\|\s*$", re.MULTILINE)
    
    md_files = [f for f in os.listdir(directory) if f.endswith(".md")]
    md_files.sort()
    
    print(f"Scanning {len(md_files)} markdown files in {directory}...\n")
    
    files_with_tables = []
    
    for filename in md_files:
        filepath = os.path.join(directory, filename)
        
        try:
            with open(filepath, "r", encoding="utf-8") as f:
                content = f.read()
            
            lines = content.split("\n")
            table_lines = []
            
            for i, line in enumerate(lines, 1):
                if table_pattern.match(line):
                    table_lines.append(i)
            
            if table_lines:
                files_with_tables.append((filename, table_lines))
                print(f"📊 {filename}:")
                print(f"   Tables found at lines: {table_lines}")
                print(f"   Total table lines: {len(table_lines)}")
                print()
                
        except Exception as e:
            print(f"❌ Error reading {filename}: {e}")
    
    print(f"\nSummary: {len(files_with_tables)} out of {len(md_files)} files contain tables")
    
    return files_with_tables

# Scan the teradataml_user_guide directory
table_files = find_tables_in_markdown_files("teradataml_user_guide")


Scanning 36 markdown files in teradataml_user_guide...

📊 01_Introduction_to_Teradata_Package_for_Python.md:
   Tables found at lines: [99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157]
   Total table lines: 59

📊 04_DataFrames_Setup_and_Basics_(Sources_Non_Default_DB_UAF).md:
   Tables found at lines: [30, 31, 32, 33, 34, 35, 36, 37, 38]
   Total table lines: 9

📊 05_DataFrame_Manipulation_(Core_API).md:
   Tables found at lines: [407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 975, 976, 977, 978, 979, 980, 981, 982, 983, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1258, 1259, 126

In [9]:
# Run consolidation on generated markdown
total_changed = consolidate_md_tables("teradataml_user_guide")
print(f'\n✅ Consolidation complete. {total_changed} file(s) changed.')

✓ Merged tables in: 01_Introduction_to_Teradata_Package_for_Python.md
✓ Merged tables in: 04_DataFrames_Setup_and_Basics_(Sources_Non_Default_DB_UAF).md
✓ Merged tables in: 05_DataFrame_Manipulation_(Core_API).md
✓ Merged tables in: 07_Executing_Python_Functions_Inside_Database_Engine_20.md
✓ Merged tables in: 08_teradataml_DataFrame_Column.md
✓ Merged tables in: 09_teradataml_Window_Aggregates.md
✓ Merged tables in: 16_BYOM_(Bring_Your_Own_Model)_Management.md
✓ Merged tables in: 17_Working_with_Geospatial_Data.md
✓ Merged tables in: 25_Additional_Information.md
✓ Merged tables in: 25_Data_Type_Mapping.md
✓ Merged tables in: 25_teradataml_Extension_with_SQLAlchemy.md
✓ Merged tables in: 25_Teradata_Package_for_Python_Limitations_and_Considerations.md
✓ Merged tables in: A_Teradata_Package_for_Python_Limitations_and_Considerations.md
✓ Merged tables in: C_teradataml_Extension_with_SQLAlchemy.md
✓ Merged tables in: D_Data_Type_Mapping.md
✓ Merged tables in: E_Additional_Information.md

